In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
print(sys.path)
from src.utils import *
from src.calcEMA import *
from src.robo import *
from src.myenv import *
import plotly.express as px

['/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader/src/jupyter_nb', '/usr/lib64/python310.zip', '/usr/lib64/python3.10', '/usr/lib64/python3.10/lib-dynload', '', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages/IPython/extensions', '/home/marcelo/.ipython']


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
# lightgbm  xgboost
estimator = 'lr'
_compare_models = False

start_train_date = '2023-06-01'  # train < and test >=
start_test_date = '2023-08-08'  # train < and test >=

stop_loss = 2.0
label = 'status'
regression_times = 24 * 30 * 2  # horas
regression_profit_and_loss = 24
# numeric_features=['open', 'high', 'low', 'volume', 'close', 'rsi']
numeric_features = ['open', 'high', 'low', 'volume', 'close' ]
calc_rsi = True


### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [3]:
use_cols = date_features + numeric_features
print('start_train_engine: use cols: ', use_cols)
print(f'start_train_engine: reading data - start date: {start_train_date}...')
all_data = get_data(symbol, save_database=False, interval='1h', tail=-1, adjust_index=True, columns=use_cols)
print('start_train_engine: get_data  all_data duplicated: ', all_data.index.duplicated().sum())

all_data = all_data[(all_data['open_time'] >= start_train_date)]  # .copy()
print('start_train_engine: info after reading data: ')
all_data.info()
print('start_train_engine: filter start_train_date all_data duplicated: ', all_data.index.duplicated().sum())

if calc_rsi:
    print('start_train_engine: calculating RSI...')
    all_data = calc_RSI(all_data)
    numeric_features.append('rsi')
    all_data.dropna(inplace=True)
print('start_train_engine: info after calculating RSI: ')
all_data.info()
print('start_train_engine: all_data duplicated: ', all_data.index.duplicated().sum())

print('start_train_engine: calculating regresstion_times...')
all_data, features_added = regresstion_times(all_data, numeric_features, regression_times, last_one=False)
print('start_train_engine: info after calculating regresstion_times: ')
all_data.info()
print('start_train_engine: all_data duplicated: ', all_data.index.duplicated().sum())

start_train_engine: use cols:  ['open_time', 'open', 'high', 'low', 'volume', 'close']
start_train_engine: reading data - start date: 2023-06-01...
get_database: name:  /home/marcelo/des/mg_crypto_trader/src/data/BTCUSDT/BTCUSDT.csv
get_database: count_rows: 52910 - symbol: BTCUSDT - tail: -1 - adjust_index: True
get_database: duplicated: 0
get_data: Downloading data for symbol: BTCUSDT
get_data: Max date database:  2023-09-07 22:00:00
start_train_engine: get_data  all_data duplicated:  0
start_train_engine: info after reading data: 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2375 entries, 2023-06-01 00:00:00 to 2023-09-07 22:00:00
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   open_time  2375 non-null   datetime64[ns]
 1   open       2375 non-null   float32       
 2   high       2375 non-null   float32       
 3   low        2375 non-null   float32       
 4   volume     2375 non-null   floa

In [4]:
print(f'start_train_engine: Filtering test_data: start_test_date: {start_test_date}')
test_data = all_data[all_data['open_time'] >= start_test_date]
print('start_train_engine: info after filtering test_data: ')
test_data.info()
print('start_train_engine: test_data duplicated: ', test_data.index.duplicated().sum())

start_train_engine: Filtering test_data: start_test_date: 2023-08-08
start_train_engine: info after filtering test_data: 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 743 entries, 2023-08-08 00:00:00 to 2023-09-07 22:00:00
Columns: 8648 entries, open_time to rsi_1440
dtypes: category(1), datetime64[ns](1), float32(8646)
memory usage: 24.5 MB
start_train_engine: test_data duplicated:  0


In [5]:
experiment, model = load_model(symbol, estimator)  # cassification_experiment

load_model: Loading model: BTCUSDT_lr_SL_2.0_RT_1440_RPL_24_2
load_model: Model obj: Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(transformer=LabelEncoder())),
                ('date_feature_extractor',
                 TransformerWrapper(include=['open_time'],
                                    transformer=ExtractDateTimeFeatures(features=['hour',
                                                                                  'day',
                                                                                  'month']))),
                ('numerical_imputer',
                 TransformerWrapper(include=['open', 'high', 'low', 'volume',
                                             '...
                 TransformerWrapper(include=['symbol'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 Transformer

In [6]:
predict = experiment.predict_model(model, data=test_data)
predict = predict.sort_values(date_features)

In [7]:
def test_trading_crypto_v3(data: pd.DataFrame, start_date, end_date, value: float, stop_loss = 3.0):
  _data = data.copy()
  _data.index = _data['open_time']
  _data = _data[(_data.index >= start_date) & (_data.index <= end_date)]
  saldo = value
  operacao = ''
  comprado = False
  valor_compra = 0 
  valor_venda = 0
  diff = 0.0
  
  operacao_compra = ''
  for row_nu in range(1, _data.shape[0]):
    open_time = pd.to_datetime(_data.iloc[row_nu:row_nu+1]['open_time'].values[0]).strftime("%Y-%m-%d %Hh")
    type(open_time)
    operacao = _data.iloc[row_nu:row_nu+1]['prediction_label'].values[0]
    #print(operacao)
    if (operacao.startswith('SOBE') or operacao.startswith('CAI')) and not comprado:
      operacao_compra = operacao
      valor_compra = round(_data.iloc[row_nu:row_nu+1]['close'].values[0], 2)
      print(f'[{row_nu}][{operacao_compra}][{open_time}] => Compra: {valor_compra}')
      comprado = True

    if comprado:
      diff = 100 * (_data.iloc[row_nu:row_nu+1]['close'].values[0] - valor_compra) / valor_compra
      #print(f'[{row_nu}][{operacao_compra}][{open_time}] Diff ==> {round(diff,2)}% - Comprado: {comprado}')
    
    if (abs(diff) >= stop_loss) and comprado:
      valor_venda = round(_data.iloc[row_nu:row_nu+1]['close'].values[0],2)
      if operacao_compra.startswith('SOBE'):
        saldo += round(saldo * (diff/100), 2)
      else:
        saldo += round(saldo * (-diff/100), 2)
      print(f'[{row_nu}][{operacao_compra}][{open_time}] => Venda: {valor_venda} => Diff: {round(diff,2)}% ==> Saldo: {saldo}')
      comprado = False

  print(f'Saldo: {saldo}')
  return saldo

print('Min Data: ', predict['open_time'].min())
print('Max Data: ', predict['open_time'].max())
test_trading_crypto_v3(predict, '2023-01-01', predict['open_time'].max(), 100.0, stop_loss)

Min Data:  2023-08-08 00:00:00
Max Data:  2023-09-07 22:00:00
[1][SOBE_2.0][2023-08-08 01h] => Compra: 29151.859375
[16][SOBE_2.0][2023-08-08 16h] => Venda: 29748.0 => Diff: 2.04% ==> Saldo: 102.04
[53][SOBE_2.0][2023-08-10 05h] => Compra: 29585.83984375
[212][SOBE_2.0][2023-08-16 20h] => Venda: 28974.009765625 => Diff: -2.07% ==> Saldo: 99.93
[214][SOBE_2.0][2023-08-16 22h] => Compra: 28910.0
[231][SOBE_2.0][2023-08-17 15h] => Venda: 27934.01953125 => Diff: -3.38% ==> Saldo: 96.56
[232][SOBE_2.0][2023-08-17 16h] => Compra: 27937.349609375
[237][SOBE_2.0][2023-08-17 21h] => Venda: 26253.919921875 => Diff: -6.03% ==> Saldo: 90.74000000000001
[238][SOBE_2.0][2023-08-17 22h] => Compra: 26801.2890625
[253][SOBE_2.0][2023-08-18 13h] => Venda: 26241.779296875 => Diff: -2.09% ==> Saldo: 88.85000000000001
[254][SOBE_2.0][2023-08-18 14h] => Compra: 26012.080078125
[377][SOBE_2.0][2023-08-23 17h] => Venda: 26539.33984375 => Diff: 2.03% ==> Saldo: 90.65
[378][SOBE_2.0][2023-08-23 18h] => Compra: 

89.71